# 차원축소 Assignment1

## 1) PCA의 과정 차근차근 밟아보기

In [1]:
#기본 모듈 불러오기
import numpy as np
import numpy.linalg as lin
import matplotlib.pyplot as plt
import pandas as pd
import random

In [2]:
#설명변수 x1, x2, x3
x1 = [95, 91, 66, 94, 68, 63, 12, 73, 93, 51, 13, 70, 63, 63, 97, 56, 67, 96, 75, 6]
x2 = [56, 27, 25, 1, 9, 80, 92, 69, 6, 25, 83, 82, 54, 97, 66, 93, 76, 59, 94, 9]
x3 = [57, 34, 9, 79, 4, 77, 100, 42, 6, 96, 61, 66, 9, 25, 84, 46, 16, 63, 53, 30]

In [3]:
#   설명변수들을 하나의 행렬로
X = np.stack((x1,x2,x3),axis=0)

In [4]:
X = pd.DataFrame(X.T,columns=['x1','x2','x3'])

In [5]:
X

,x1,x2,x3
0,95,56,57
1,91,27,34
2,66,25,9
3,94,1,79
4,68,9,4
5,63,80,77
6,12,92,100
7,73,69,42
8,93,6,6
9,51,25,96


### 1-1) 먼저 PCA를 시작하기 전 데이터 scaling!! 중요합니다!

https://datascienceschool.net/03%20machine%20learning/02.01%20%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EC%A0%84%EC%B2%98%EB%A6%AC%20%EA%B8%B0%EC%B4%88.html?highlight=%EC%A0%84%EC%B2%98%EB%A6%AC  를 참고하시면 도움이 될거에요

* 실제 데이터에선 측정 단위의 영향을 받음 : **표준화 사용**

$\Rightarrow $ 각 변수가 평균 0, 표준편차가 1이 되도록 표준화 

`StandardScaler` : 스케일링 및 변수변환

1. 학습용 데이터를 입력으로 `fit()` 메서드를 실행하면 평균값과 표준편차를 계산하여 객체내에 저장
2. 다시 학습용 데이터를 입력으로 하여 `transform()` 메서드를 실행하면 저장했던 평균값을 빼서 새로운 평균값이 0이 되도록 만들고, 저장한 표분편차로 나우어 새로운 표준편차가 1이 되도록 데이터를 변환하여 출력함. 1단계와 2단계를 합쳐서 `fit_transform()` 메서드를 실행할 수도 있음.
3. 검증용 데이터를 입력으로 하여 `transform()` 메서드를 실행해도 학습용 데이터의 평균값과 표준편차를 사용하여 검증용 데이터를 변환

In [6]:
#scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_std = scaler.fit_transform(X)  #sclaing 후 데이터

In [7]:
X_std

array([[ 1.08573604,  0.02614175,  0.30684189],
       [ 0.93801686, -0.86575334, -0.46445467],
       [ 0.01477192, -0.92726334, -1.30282049],
       [ 1.04880625, -1.66538341,  1.04460382],
       [ 0.08863151, -1.41934339, -1.47049366],
       [-0.09601747,  0.76426183,  0.97753455],
       [-1.97943714,  1.13332186,  1.74883111],
       [ 0.2732805 ,  0.42595679, -0.1961776 ],
       [ 1.01187645, -1.5116084 , -1.40342439],
       [-0.53917504, -0.92726334,  1.61469258],
       [-1.94250735,  0.85652683,  0.44098042],
       [ 0.16249111,  0.82577183,  0.60865359],
       [-0.09601747, -0.03536825, -1.30282049],
       [-0.09601747,  1.28709688, -0.76626636],
       [ 1.15959564,  0.33369178,  1.21227698],
       [-0.35452606,  1.16407687, -0.06203907],
       [ 0.05170172,  0.64124181, -1.06807806],
       [ 1.12266584,  0.11840676,  0.50804969],
       [ 0.3471401 ,  1.19483187,  0.17270336],
       [-2.20101593, -1.41934339, -0.5985932 ]])

In [8]:
# X_std의 전치행렬
features = X_std.T 

In [9]:
features

array([[ 1.08573604,  0.93801686,  0.01477192,  1.04880625,  0.08863151,
        -0.09601747, -1.97943714,  0.2732805 ,  1.01187645, -0.53917504,
        -1.94250735,  0.16249111, -0.09601747, -0.09601747,  1.15959564,
        -0.35452606,  0.05170172,  1.12266584,  0.3471401 , -2.20101593],
       [ 0.02614175, -0.86575334, -0.92726334, -1.66538341, -1.41934339,
         0.76426183,  1.13332186,  0.42595679, -1.5116084 , -0.92726334,
         0.85652683,  0.82577183, -0.03536825,  1.28709688,  0.33369178,
         1.16407687,  0.64124181,  0.11840676,  1.19483187, -1.41934339],
       [ 0.30684189, -0.46445467, -1.30282049,  1.04460382, -1.47049366,
         0.97753455,  1.74883111, -0.1961776 , -1.40342439,  1.61469258,
         0.44098042,  0.60865359, -1.30282049, -0.76626636,  1.21227698,
        -0.06203907, -1.06807806,  0.50804969,  0.17270336, -0.5985932 ]])

### 1-2) 다음으로 공분산 행렬을 구합니다

https://docs.scipy.org/doc/numpy/reference/generated/numpy.cov.html 를 참고하시면 도움이 될거에요

`numpy.cov` : Estimate a covariance matrix, given data and weights.

* numpy.cov(m, y=None, rowvar=True, bias=False, ddof=None, fweights=None, aweights=None, *, dtype=None)

In [10]:
cov_matrix = np.cov(features)

In [11]:
cov_matrix

array([[ 1.05263158, -0.2037104 , -0.12079228],
       [-0.2037104 ,  1.05263158,  0.3125801 ],
       [-0.12079228,  0.3125801 ,  1.05263158]])

### 1-3) 이제 고유값과 고유벡터를 구합니다

실습코드를 참고해주세요

In [12]:
import numpy.linalg as lin

eigenvalues = lin.eig(cov_matrix)[0] # 고유값
eigenvectors = lin.eig(cov_matrix)[1] # 고유벡터

In [13]:
print(eigenvalues)
print(eigenvectors)

[1.48756162 0.94435407 0.72597904]
[[ 0.47018528 -0.85137353 -0.23257022]
 [-0.64960236 -0.15545725 -0.74421087]
 [-0.59744671 -0.50099516  0.62614797]]


In [14]:
mat = np.zeros((3,3))

In [15]:
mat

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [16]:
mat[0][0] = eigenvalues[0]
mat[1][1] = eigenvalues[1]
mat[2][2] = eigenvalues[2]

In [17]:
# lambda
mat

array([[1.48756162, 0.        , 0.        ],
       [0.        , 0.94435407, 0.        ],
       [0.        , 0.        , 0.72597904]])

### 1-4) 고유값 분해: 고유값 분해의 곱으로 원래 공분산 행렬을 구해봅니다

https://docs.scipy.org/doc/numpy/reference/generated/numpy.dot.html 를 참고해주세요

* 고유값 분해 : 정방행렬 B에 대해 $B\mathbf{V} = \lambda\mathbf{V}$를 만족하는 고유값 $\lambda$, 벡터 $\mathbf{V}$를 찾는 작업
* 즉, 선형변환 B에 의한 결과가 자기자신의 상수배가 되는, 0이 아닌 벡터(고유벡터)를 구하는 작업
* 대각화 : 고유벡터 행렬 $\mathbf{V}$로 행렬 A를 표현
$$ A = \mathbf{V}\Lambda\mathbf{V}^T$$

`numpy.dot` : Dot product of two arrays
* numpy.dot(a, b, out=None)

In [18]:
np.dot(np.dot(eigenvectors, mat), eigenvectors.T)
#1-2에서 구했던 공분산과 동일한가요?

array([[ 1.05263158, -0.2037104 , -0.12079228],
       [-0.2037104 ,  1.05263158,  0.3125801 ],
       [-0.12079228,  0.3125801 ,  1.05263158]])

### 1-5) 마지막으로 고유 벡터 축으로 값을 변환합니다


https://docs.scipy.org/doc/numpy/reference/generated/numpy.concatenate.html

`numpy.concatenate` : Join a sequence of arrays along an existing axis.
* numpy.concatenate((a1, a2, ...), axis=0, out=None, dtype=None, casting="same_kind")

In [19]:
def new_coordinates(X,eigenvectors):
    for i in range(eigenvectors.shape[0]):
        if i == 0:
            new = [X.dot(eigenvectors.T[i])]
        else:
            new = np.concatenate((new, [X.dot(eigenvectors.T[i])]), axis=0)
    return new.T
# X, eigenvector 행렬곱을 통해 데이터를 projection
# 모든 고유 벡터 축으로 데이터를 projection한 값입니다

In [20]:
new_coordinates(X_std,eigenvectors)

# 새로운 축으로 변환되어 나타난 데이터들입니다

array([[ 0.31019368, -1.08215716, -0.07983642],
       [ 1.28092404, -0.43132556,  0.13533091],
       [ 1.38766381,  0.78428014, -0.12911446],
       [ 0.95087515, -1.15737142,  1.6495519 ],
       [ 1.84222365,  0.88189889,  0.11493111],
       [-1.12563709, -0.52680338,  0.06564012],
       [-2.71174416,  0.63290138,  0.71195473],
       [-0.03100441, -0.20059783, -0.50339479],
       [ 2.29618509,  0.07661447,  0.01087174],
       [-0.61585248, -0.205764  ,  1.82651199],
       [-1.73320252,  1.29971699,  0.09045178],
       [-0.82366049, -0.57164535, -0.27123176],
       [ 0.75619512,  0.73995175, -0.76710616],
       [-0.42344386,  0.26555394, -1.41533681],
       [-0.39581307, -1.64646874,  0.24104031],
       [-0.88581498,  0.15195119, -0.82271209],
       [ 0.24587691,  0.39139878, -1.15801831],
       [ 0.14741103, -1.22874561, -0.03110396],
       [-0.7161265 , -0.56781471, -0.86180345],
       [ 0.24475107,  2.39442622,  1.19337361]])

## 2) PCA를 구현해보기

위의 과정을 하나의 함수로 표현해주세요!

1)에서 진행한 과정을 그대로 가져오면 됩니다..! , 

위에서 만든 함수(new_coordinate)도 가져오셔서 사용해도 좋아요!

+하지만 이 함수는 number이라는 인자를 받아서 pca에서 주성분 개수를 고려하도록 합니다!

고유값이 큰 축 순서로 정렬하는 과정 + 주성분 개수 number을 고려해야한다는 점을 유의해서 함수를 정의해주세요


In [21]:
from sklearn.preprocessing import StandardScaler

def MYPCA(X,number):
    # 스케일링
    scaler = StandardScaler()
    X_std = scaler.fit_transform(X)
    
    # 공분산 행렬 계산
    features = X_std.T
    cov_matrix = np.cov(features)
    
    # 고유값과 고유벡터 계산
    eigenvalues = lin.eig(cov_matrix)[0] # 고유값
    eigenvectors = lin.eig(cov_matrix)[1] # 고유벡터
    
    # 고유값 정렬 및 고유값 크기에 따라 고유벡터도 정렬
    new_coordinates(X_std, eigenvectors)
    
    new_coordinate = new_coordinates(X_std, eigenvectors)
    
    index = eigenvalues.argsort()
    index = list(index)
    
    # number만큼 주성분 축의 개수 설정
    for i in range(number):
        if i == 0:
            new = [new_coordinate[:, index.index(i)]]
        else:
            new = np.concatenate(([new_coordinate[:, index.index(i)]], new), axis=0)
    
    return new.T #새로운 축으로 변환되어 나타난 데이터 return

In [23]:
MYPCA(X,3)

# 새로운 축으로 잘 변환되어서 나타나나요?
# 위에서 했던 PCA랑은 차이가 있을 수 있어요 
# 왜냐하면 위에서는 고유값이 큰 축 순서로 정렬을 안했었거든요

array([[ 0.31019368, -1.08215716, -0.07983642],
       [ 1.28092404, -0.43132556,  0.13533091],
       [ 1.38766381,  0.78428014, -0.12911446],
       [ 0.95087515, -1.15737142,  1.6495519 ],
       [ 1.84222365,  0.88189889,  0.11493111],
       [-1.12563709, -0.52680338,  0.06564012],
       [-2.71174416,  0.63290138,  0.71195473],
       [-0.03100441, -0.20059783, -0.50339479],
       [ 2.29618509,  0.07661447,  0.01087174],
       [-0.61585248, -0.205764  ,  1.82651199],
       [-1.73320252,  1.29971699,  0.09045178],
       [-0.82366049, -0.57164535, -0.27123176],
       [ 0.75619512,  0.73995175, -0.76710616],
       [-0.42344386,  0.26555394, -1.41533681],
       [-0.39581307, -1.64646874,  0.24104031],
       [-0.88581498,  0.15195119, -0.82271209],
       [ 0.24587691,  0.39139878, -1.15801831],
       [ 0.14741103, -1.22874561, -0.03110396],
       [-0.7161265 , -0.56781471, -0.86180345],
       [ 0.24475107,  2.39442622,  1.19337361]])

## 3) sklearn의 PCA 패키지와 비교해보기

https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html 를 참고하시면 도움이 될거에요

* class sklearn.decomposition.PCA(n_components=None, *, copy=True, whiten=False, svd_solver='auto', tol=0.0, iterated_power='auto', random_state=None)

In [26]:
from sklearn.decomposition import PCA

# 주성분 갯수 : 3
pca = PCA(n_components=3)
# Scaling된 데이터인 X_Std로 PCA모델에 fit_transform
pca_scaled = pca.fit_transform(X_std)

In [28]:
pca_scaled

array([[-0.31019368, -1.08215716, -0.07983642],
       [-1.28092404, -0.43132556,  0.13533091],
       [-1.38766381,  0.78428014, -0.12911446],
       [-0.95087515, -1.15737142,  1.6495519 ],
       [-1.84222365,  0.88189889,  0.11493111],
       [ 1.12563709, -0.52680338,  0.06564012],
       [ 2.71174416,  0.63290138,  0.71195473],
       [ 0.03100441, -0.20059783, -0.50339479],
       [-2.29618509,  0.07661447,  0.01087174],
       [ 0.61585248, -0.205764  ,  1.82651199],
       [ 1.73320252,  1.29971699,  0.09045178],
       [ 0.82366049, -0.57164535, -0.27123176],
       [-0.75619512,  0.73995175, -0.76710616],
       [ 0.42344386,  0.26555394, -1.41533681],
       [ 0.39581307, -1.64646874,  0.24104031],
       [ 0.88581498,  0.15195119, -0.82271209],
       [-0.24587691,  0.39139878, -1.15801831],
       [-0.14741103, -1.22874561, -0.03110396],
       [ 0.7161265 , -0.56781471, -0.86180345],
       [-0.24475107,  2.39442622,  1.19337361]])

In [29]:
MYPCA(X,3)

array([[ 0.31019368, -1.08215716, -0.07983642],
       [ 1.28092404, -0.43132556,  0.13533091],
       [ 1.38766381,  0.78428014, -0.12911446],
       [ 0.95087515, -1.15737142,  1.6495519 ],
       [ 1.84222365,  0.88189889,  0.11493111],
       [-1.12563709, -0.52680338,  0.06564012],
       [-2.71174416,  0.63290138,  0.71195473],
       [-0.03100441, -0.20059783, -0.50339479],
       [ 2.29618509,  0.07661447,  0.01087174],
       [-0.61585248, -0.205764  ,  1.82651199],
       [-1.73320252,  1.29971699,  0.09045178],
       [-0.82366049, -0.57164535, -0.27123176],
       [ 0.75619512,  0.73995175, -0.76710616],
       [-0.42344386,  0.26555394, -1.41533681],
       [-0.39581307, -1.64646874,  0.24104031],
       [-0.88581498,  0.15195119, -0.82271209],
       [ 0.24587691,  0.39139878, -1.15801831],
       [ 0.14741103, -1.22874561, -0.03110396],
       [-0.7161265 , -0.56781471, -0.86180345],
       [ 0.24475107,  2.39442622,  1.19337361]])

두 결과를 비교해보면 절대값은 동일하나 부호가 다른 경우가 있음을 알 수 있음.

## 4) PCA 외 수업에서 다룬 차원축소 기법(FS,GA,LDA,SNE 등) 중 택1하여 밑바닥부터 만들어보기

## FS

* 모델 학습에 필요한 features을 선별하는 작업
* feature가 너무 많으면 overfitting의 위험이 있기 때문에 적절한 feature를 사용하는 것이 중요함.

**1.Wrapper method**
* 예측 모델을 사용해서 feature의 부분집합을 만들어 시행하면서 최적화된 집합을 찾는 방법이나 많은 시간과 비용이 소요됨.

**2.Fiter method**
* 통계적 측정 방법을 사용하여 feature들의 상관관계를 알아내는 방법
* 도움이 되지 않는 feature들은 걸러내는 방식

**3.Embedded method**
* wrapper와 filter을 결합한 방법

## data import & stratified sampling

In [137]:
import pandas as pd
import numpy as np
import gc
import warnings
import re
warnings.filterwarnings('ignore')

In [138]:
application_train = pd.read_csv('application_train.csv')
application_train = application_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
application_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [139]:
# Stratified Sampling
application_sample1 = application_train.loc[application_train.TARGET==1].sample(frac=0.1, replace=False)
print('label 1 sample size:', str(application_sample1.shape[0]))
application_sample0 = application_train.loc[application_train.TARGET==0].sample(frac=0.1, replace=False)
print('label 0 sample size:', str(application_sample0.shape[0]))
application = pd.concat([application_sample1, application_sample0], axis=0).sort_values('SK_ID_CURR')                   

label 1 sample size: 2482
label 0 sample size: 28269


In [140]:
# Impute missing values
categorical_list = []
numerical_list = []
for i in application.columns.tolist():
    if application[i].dtype=='object':
        categorical_list.append(i)
    else:
        numerical_list.append(i)
print('Number of categorical features:', str(len(categorical_list)))
print('Number of numerical features:', str(len(numerical_list)))

Number of categorical features: 16
Number of numerical features: 106


In [141]:
from sklearn.impute import SimpleImputer
application[numerical_list] = SimpleImputer(strategy='median').fit_transform(application[numerical_list])

In [142]:
# Deal with Categorical features: OneHotEncoding
del application_train; gc.collect()
application = pd.get_dummies(application, drop_first=True)
print(application.shape)

(30751, 227)


In [143]:
# Feature matrix and target
X = application.drop(['SK_ID_CURR', 'TARGET'], axis=1)
y = application.TARGET
feature_name = X.columns.tolist()

우리의 목표는 230개의 피처에서 100개만을 고르는 것임.

* xxx_support : 이 피쳐를 선택했는지 아닌지 알려주는 리스트
* xxx_feature : xxx_list에서 상관계수가 높았던 상위 100개의 피쳐를 X에서 인덱스로 찾은 후 xxx_feature 리스트로 저장

### 1. Wrapper Method

* RFE는 점점 피쳐 부분 집합의 개수를 줄여가면서 재귀적으로 피쳐를 찾는 것이 목적임. 
* 모델이 처음의 피쳐 집합으로 학습한 후 중요도를 계산한 후, 중요도가 덜한 피쳐를 제거하고 반복함.

In [122]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
rfe_selector = RFE(estimator=LogisticRegression(), n_features_to_select=100, step=10, verbose=5)
rfe_selector.fit(X_norm, y)

Fitting estimator with 226 features.
Fitting estimator with 216 features.
Fitting estimator with 206 features.
Fitting estimator with 196 features.
Fitting estimator with 186 features.
Fitting estimator with 176 features.
Fitting estimator with 166 features.
Fitting estimator with 156 features.
Fitting estimator with 146 features.
Fitting estimator with 136 features.
Fitting estimator with 126 features.
Fitting estimator with 116 features.
Fitting estimator with 106 features.


RFE(estimator=LogisticRegression(), n_features_to_select=100, step=10,
    verbose=5)

In [123]:
rfe_support = rfe_selector.get_support()
rfe_feature = X.loc[:, rfe_support].columns.tolist()
print(str(len(rfe_feature)), 'selected features')

100 selected features


### 2. Filter Method

### 2.1 Pearson Correlation

In [124]:
# 피어슨 상관계수는 따로 support 와 feature 를 반환하는 함수가 없기 때문에 직접 만들어주었음.
def cor_selector(X, y):
    cor_list = []
    # calculate the correlation with y for each feature
    for i in X.columns.tolist():
        cor = np.corrcoef(X[i], y)[0, 1]
        cor_list.append(cor)
    # replace NaN with 0
    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    # feature name
    cor_feature = X.iloc[:,np.argsort(np.abs(cor_list))[-100:]].columns.tolist()
    # feature selection? 0 for not select, 1 for select
    cor_support = [True if i in cor_feature else False for i in feature_name]
    return cor_support, cor_feature

In [125]:
cor_support, cor_feature = cor_selector(X, y)
print(str(len(cor_feature)), 'selected features')

100 selected features


### 2.2 Chi-2

카이제곱은 정규분포에 적용해야 하기 때문에 `MinMaxScaler`로 정규화 해주어야 함.

In [126]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

In [121]:
X_norm = MinMaxScaler().fit_transform(X)
chi_selector = SelectKBest(chi2, k=100)
chi_selector.fit(X_norm, y)

SelectKBest(k=100, score_func=<function chi2 at 0x000001958193A1F0>)

In [127]:
chi_support = chi_selector.get_support()
chi_feature = X.loc[:, chi_support].columns.tolist()
print(str(len(chi_feature)), 'selected features')

100 selected features


`SelectKBest` 모듈은 다음과 같은 목적에서 해당 모델을 쓸 수 있음.
* 회귀 : f_regression, mutual_into_regression
* 분류 : chi2, f_classif, mutual_into_classif

### 3. Embedded Method

* sklearn에서 `SelectFromModel`이라는 모듈을 사용함. 모델 낼에서 바로 피쳐를 선택함.
* L1, RandomForest 2개의 모델 사용

### 3.1 L1

In [128]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

In [129]:
embedded_lr_selector = SelectFromModel(LogisticRegression(penalty='l1', solver='liblinear'), '1.25*median')
embedded_lr_selector.fit(X_norm, y)

SelectFromModel(estimator=LogisticRegression(penalty='l1', solver='liblinear'),
                threshold='1.25*median')

In [130]:
embedded_lr_support = embedded_lr_selector.get_support()
embedded_lr_feature = X.loc[:, embedded_lr_support].columns.tolist()
print(str(len(embedded_lr_feature)), 'selected features')

107 selected features


### 3.2 Random Forest

In [131]:
from sklearn.ensemble import RandomForestClassifier

In [132]:
embedded_rf_selector = SelectFromModel(RandomForestClassifier(n_estimators=100), threshold='1.25*median')
embedded_rf_selector.fit(X, y)

SelectFromModel(estimator=RandomForestClassifier(), threshold='1.25*median')

In [133]:
embedded_rf_support = embedded_rf_selector.get_support()
embedded_rf_feature = X.loc[:, embedded_rf_support].columns.tolist()
print(str(len(embedded_rf_feature)), 'selected features')

96 selected features


## Summary

In [156]:
X = application.drop(['SK_ID_CURR'], axis=1)
feature_name = X.columns.tolist()

In [157]:
pd.set_option('display.max_rows', None)
# put all selection together
feature_selection_df = pd.DataFrame({'Feature':feature_name, 'Pearson':cor_support, 'Chi-2':chi_support, 'RFE':rfe_support, 'Logistics':embedded_lr_support,
                                    'Random Forest':embedded_rf_support})
# count the selected times for each feature
feature_selection_df['Total'] = np.sum(feature_selection_df, axis=1)
# display the top 100
feature_selection_df = feature_selection_df.sort_values(['Total','Feature'] , ascending=False)
feature_selection_df.index = range(1, len(feature_selection_df)+1)
feature_selection_df.head(100)

,Feature,Pearson,Chi-2,RFE,Logistics,Random Forest,Total
1,OBS_60_CNT_SOCIAL_CIRCLE,True,True,True,True,True,5
2,NAME_HOUSING_TYPE_With parents,True,True,True,True,True,5
3,NAME_HOUSING_TYPE_House / apartment,True,True,True,True,True,5
4,NAME_CONTRACT_TYPE_Revolving loans,True,True,True,True,True,5
5,LIVINGAPARTMENTS_AVG,True,True,True,True,True,5
6,LIVE_CITY_NOT_WORK_CITY,True,True,True,True,True,5
7,FLAG_DOCUMENT_2,True,True,True,True,True,5
8,EXT_SOURCE_2,True,True,True,True,True,5
9,EXT_SOURCE_1,True,True,True,True,True,5
10,ENTRANCES_MEDI,True,True,True,True,True,5
